# ['abs_chng_timestamp_1', +
 'abs_chng_timestamp_2', +
 'abs_chng_timestamp_3', +
 
 'content_he_mean_answered_correctly', + 
 'content_he_part_count_answered_correctly', +
 'content_mean_answered_correctly', + 
 'content_sum_answered_correctly', +
 
 'part_bundle_id',  = "part" * 100000 + "bundle_id" + 
 
 'prior_question_elapsed_time', +
 
 
 'user_he_mean_answered_correctly', +
 'user_mean_answered_correctly', +
 'user_sum_answered_correctly' +
 'user_he_mean_prior_question_elapsed_time', +
 
 'user_he_ucount_part', +
 
 'user_ema_answered_correctly', +
 'user_he_ema_answered_correctly', +
 
 'attempt_no_count', +
 'attempt_no_mean', +
 
 'strike', +
 'strike_bundle', +
 
 'lag_expl_and_part_bool', ("lag_part" == "part") & "prior_question_had_explanation"+ 
 
 
 'mul_user_content_ema_answ', = "user_ema_answered_correctly" * "content_mean_answered_correctly",
 
 'rel_user_content_he_mean_answ', = "user_he_mean_answered_correctly" / "content_he_mean_answered_correctly",
 'rel_user_he_mean_answ', = "user_he_mean_answered_correctly" / "user_mean_answered_correctly",

 'trend_user_he_mean_answ', = "user_he_ema_answered_correctly" / "user_he_mean_answered_correctly",
 ]

In [1]:
import pandas as pd
from pathlib import Path
from typing import List, Dict, Tuple
import numpy as np
pd.options.display.float_format = '{:.5f}'.format

In [2]:
result_fillna = dict()
result_features = dict()

In [3]:
def load_and_preprocess_vitrine() -> pd.DataFrame:
    vitrine = pd.read_csv(Path("../data/all_features_ns_full/vitrine.csv"))

    vitrine = vitrine[vitrine.content_type_id == 0].reset_index(drop=True)
    vitrine["prior_question_had_explanation"].fillna(False, inplace=True)
    vitrine["prior_question_had_explanation"] = list(
        map(lambda x: int(x), list(vitrine["prior_question_had_explanation"]))
    )

    return vitrine


def aggregate(
    df_i: pd.DataFrame,
    groupby_list: List[str],
    agg_func: List,
    name_list: List,
    target_field: str,
) -> pd.DataFrame:
    df = df_i.copy()
    df = df.sort_values(by=groupby_list + ["timestamp"])
    df = df[groupby_list + [target_field]].groupby(groupby_list).agg(agg_func)
    df.columns = list(map(lambda x: x + target_field, name_list))
    return df.reset_index()


def get_user_last_f(
    df_i: pd.DataFrame, keys: List[str], nec_field: List[str]
) -> pd.DataFrame:
    df = df_i.copy()

    df = df[keys + list(set(nec_field) - set(["timestamp"])) + ["timestamp"]]
    df = df.sort_values(by=["user_id", "timestamp"])
    df = (
        df.drop_duplicates(subset=keys, keep="last")
        .sort_values(by=keys)
        .reset_index(drop=True)
    )

    return df


def get_answered_as_arr(
    vitrine: pd.DataFrame,
):
    features_user = vitrine.sort_values(by=["user_id", "timestamp"]).reset_index(
        drop=True
    )
    features_user = features_user[["user_id", "answered_correctly"]]

    return (
        features_user.groupby("user_id")["answered_correctly"]
        .apply(np.array)
        .to_frame()
    ).reset_index()


def df_features_to_dict(df: pd.DataFrame, index_fields: List[str]) -> Dict:
    df = df.set_index(index_fields)
    return df.to_dict('index')

def get_contnt_prod(
    result_fillna: Dict, 
    df: pd.DataFrame, 
    prename: str,
    keys: List[str],
    target: str = 'answered_correctly'
) -> Tuple[Dict, Dict]:
    df[f'{prename}_mean_{target}'] = df[f'{prename}_sum_{target}'] /\
                                                df[f'{prename}_count_{target}']

    result_fillna[f'{prename}_mean_{target}'] = df[f'{prename}_mean_{target}'].median()
    result_fillna[f'{prename}_sum_{target}'] = df[f'{prename}_sum_{target}'].median()
    result_fillna[f'{prename}_count_{target}'] = df[f'{prename}_count_{target}'].median()
    print(result_fillna)

    df.drop([f'{prename}_mean_{target}'], axis = 1, inplace = True)

    dict_ = df_features_to_dict(
        df,
        keys
    )
    print(list(dict_.items())[:4], len(dict_))
    del df
    return dict_, result_fillna


def get_answered_as_arr(
    vitrine: pd.DataFrame,
    agg_fields: List[str]
):
    vitrine = vitrine.sort_values(by = ['user_id', 'id']).reset_index(drop=True)
    return vitrine.groupby(agg_fields)['answered_correctly'].apply(np.array).to_frame().reset_index()




from functools import reduce
def ema(series):
    N = len(series)
    alpha = 2 / (N + 1)
    return reduce(lambda sum_, curr: (1 - alpha) * sum_ + alpha * curr, series)



vitrine = load_and_preprocess_vitrine()

'content_he_mean_answered_correctly', + 

'content_mean_answered_correctly',
'content_sum_answered_correctly',

'content_he_part_count_answered_correctly',

In [4]:
content_he_mean_answered_correctly = aggregate(
    vitrine,
    groupby_list=["content_id", "prior_question_had_explanation"],
    agg_func=["sum", "count"],
    name_list=["content_he_sum_", "content_he_count_"],
    target_field="answered_correctly",
)

content_mean_answered_correctly = aggregate(
    vitrine,
    groupby_list=["content_id"],
    agg_func=["sum", "count"],
    name_list=["content_sum_", "content_count_"],
    target_field="answered_correctly",
)

content_he_part_count_answered_correctly = aggregate(
    vitrine,
    groupby_list=["part", "prior_question_had_explanation"],
    agg_func=["sum", "count"],
    name_list=["content_he_part_sum_", "content_he_part_count_"],
    target_field="answered_correctly",
)

In [5]:
content_he_mean_answered_correctly, result_fillna = get_contnt_prod(
    result_fillna, 
    content_he_mean_answered_correctly, 
    prename = 'content_he',
    keys = ['content_id', 'prior_question_had_explanation']
)

{'content_he_mean_answered_correctly': 0.7221108884435538, 'content_he_sum_answered_correctly': 323.0, 'content_he_count_answered_correctly': 525.0}
[((0, 0), {'content_he_sum_answered_correctly': 299, 'content_he_count_answered_correctly': 360}), ((0, 1), {'content_he_sum_answered_correctly': 5967, 'content_he_count_answered_correctly': 6543}), ((1, 0), {'content_he_sum_answered_correctly': 118, 'content_he_count_answered_correctly': 145}), ((1, 1), {'content_he_sum_answered_correctly': 6471, 'content_he_count_answered_correctly': 7253})] 26905


In [6]:
content_mean_answered_correctly, result_fillna = get_contnt_prod(
    result_fillna, 
    content_mean_answered_correctly, 
    prename = 'content',
    keys = ["content_id"]
)

{'content_he_mean_answered_correctly': 0.7221108884435538, 'content_he_sum_answered_correctly': 323.0, 'content_he_count_answered_correctly': 525.0, 'content_mean_answered_correctly': 0.7342154131847726, 'content_sum_answered_correctly': 3236.0, 'content_count_answered_correctly': 4732.0}
[(0, {'content_sum_answered_correctly': 6266, 'content_count_answered_correctly': 6903}), (1, {'content_sum_answered_correctly': 6589, 'content_count_answered_correctly': 7398}), (2, {'content_sum_answered_correctly': 24890, 'content_count_answered_correctly': 44905}), (3, {'content_sum_answered_correctly': 17906, 'content_count_answered_correctly': 22973})] 13523


In [7]:
content_he_part_count_answered_correctly, result_fillna = get_contnt_prod(
    result_fillna, 
    content_he_part_count_answered_correctly, 
    prename = 'content_he_part',
    keys = ["part", "prior_question_had_explanation"]
)

{'content_he_mean_answered_correctly': 0.7221108884435538, 'content_he_sum_answered_correctly': 323.0, 'content_he_count_answered_correctly': 525.0, 'content_mean_answered_correctly': 0.7342154131847726, 'content_sum_answered_correctly': 3236.0, 'content_count_answered_correctly': 4732.0, 'content_he_part_mean_answered_correctly': 0.6309980073399225, 'content_he_part_sum_answered_correctly': 2117852.5, 'content_he_part_count_answered_correctly': 3642251.0}
[((1, 0), {'content_he_part_sum_answered_correctly': 690405, 'content_he_part_count_answered_correctly': 1108120}), ((1, 1), {'content_he_part_sum_answered_correctly': 4863491, 'content_he_part_count_answered_correctly': 6346450}), ((2, 0), {'content_he_part_sum_answered_correctly': 530985, 'content_he_part_count_answered_correctly': 832410}), ((2, 1), {'content_he_part_sum_answered_correctly': 12752354, 'content_he_part_count_answered_correctly': 17910994})] 14


In [8]:
result_features['content_he_mean_answered_correctly'] = content_he_mean_answered_correctly
result_features['content_mean_answered_correctly'] = content_mean_answered_correctly
result_features['content_he_part_count_answered_correctly'] = content_he_part_count_answered_correctly

In [9]:
result_features

{'content_he_mean_answered_correctly': {(0,
   0): {'content_he_sum_answered_correctly': 299, 'content_he_count_answered_correctly': 360},
  (0, 1): {'content_he_sum_answered_correctly': 5967,
   'content_he_count_answered_correctly': 6543},
  (1, 0): {'content_he_sum_answered_correctly': 118,
   'content_he_count_answered_correctly': 145},
  (1, 1): {'content_he_sum_answered_correctly': 6471,
   'content_he_count_answered_correctly': 7253},
  (2, 0): {'content_he_sum_answered_correctly': 2465,
   'content_he_count_answered_correctly': 5021},
  (2, 1): {'content_he_sum_answered_correctly': 22425,
   'content_he_count_answered_correctly': 39884},
  (3, 0): {'content_he_sum_answered_correctly': 656,
   'content_he_count_answered_correctly': 956},
  (3, 1): {'content_he_sum_answered_correctly': 17250,
   'content_he_count_answered_correctly': 22017},
  (4, 0): {'content_he_sum_answered_correctly': 8451,
   'content_he_count_answered_correctly': 14909},
  (4, 1): {'content_he_sum_answered_

# prior_question_elapsed_time

In [10]:
result_fillna['prior_question_elapsed_time'] = vitrine['prior_question_elapsed_time'].median()
result_fillna

{'content_he_mean_answered_correctly': 0.7221108884435538,
 'content_he_sum_answered_correctly': 323.0,
 'content_he_count_answered_correctly': 525.0,
 'content_mean_answered_correctly': 0.7342154131847726,
 'content_sum_answered_correctly': 3236.0,
 'content_count_answered_correctly': 4732.0,
 'content_he_part_mean_answered_correctly': 0.6309980073399225,
 'content_he_part_sum_answered_correctly': 2117852.5,
 'content_he_part_count_answered_correctly': 3642251.0,
 'prior_question_elapsed_time': 21000.0}

# User time features

In [11]:
for i in range(3):
    vitrine[f"lag_timestamp_{i+1}"] = vitrine.groupby(["user_id"])[
        "timestamp"
    ].shift(i + 1)
    vitrine[f"abs_chng_timestamp_{i+1}"] = abs(
        vitrine["timestamp"] - vitrine[f"lag_timestamp_{i+1}"]
    )
    result_fillna[f"abs_chng_timestamp_{i+1}"] = vitrine[f"abs_chng_timestamp_{i+1}"].median()
    
    vitrine[f"lag_timestamp_{i+1}"] = vitrine.groupby(["user_id"])[
        "timestamp"
    ].shift(i)
    df = get_user_last_f(
        vitrine,
        keys = ['user_id'], 
        nec_field = [f"lag_timestamp_{i+1}"]
    )
    result_features[f"abs_chng_timestamp_{i+1}"] = df_features_to_dict(
        df[["user_id", f"lag_timestamp_{i+1}"]],
        ['user_id']
    )
    vitrine.drop([
        f"lag_timestamp_{i+1}",
        f"abs_chng_timestamp_{i+1}",
    ], axis = 1, inplace = True)

In [12]:
vitrine[vitrine.user_id == 22318239]

id  timestamp   user_id  content_id  content_type_id  \
1039557  1060729          0  22318239        3779                0   
1039558  1060730      23082  22318239        5565                0   
1039559  1060731      37693  22318239        4535                0   
1039560  1060732      50594  22318239        3577                0   
1039561  1060733      66694  22318239        6435                0   
1039562  1060734      82594  22318239        5580                0   
1039563  1060735     105355  22318239        5708                0   
1039564  1060736     132415  22318239        9693                0   
1039565  1060737     158022  22318239        3684                0   
1039566  1060738     173207  22318239        5254                0   
1039567  1060739     242856  22318239        4388                0   
1039568  1060740     259250  22318239        6426                0   
1039569  1060741     280241  22318239        6437                0   
1039570  1060742     304518  22318239        9983                0   
1039571  1060743     328545  22318239        8219                0   
1039572  1060744     340741  22318239        5696                0   
1039573  1060745     352699  22318239        5238                0   
1039574  1060746     374622  22318239        9862                0   
1039575  1060747     398692  22318239        6151                0   
1039576  1060748     418433  22318239        5335                0   

         task_container_id  answered_correctly  prior_question_elapsed_time  \
1039557                  0                   1                          nan   
1039558                  1                   1                  12000.00000   
1039559                  2                   1                  19000.00000   
1039560                  3                   0                  12000.00000   
1039561                  4                   0                  10000.00000   
1039562                  5                   0                  13000.00000   
1039563                  6                   0                  13000.00000   
1039564                  7                   0                  20000.00000   
1039565                  8                   1                  24000.00000   
1039566                  9                   1                  23000.00000   
1039567                 10                   0                  12000.00000   
1039568                 11                   1                  13000.00000   
1039569                 12                   1                   7000.00000   
1039570                 13                   0                  12000.00000   
1039571                 14                   1                  12000.00000   
1039572                 15                   1                  11000.00000   
1039573                 16                   1                   4000.00000   
1039574                 17                   0                   5000.00000   
1039575                 18                   0                  15000.00000   
1039576                 19                   0                  17000.00000   

         prior_question_had_explanation  part  concept  intention  \
1039557                               0     5      nan        nan   
1039558                               0     5      nan        nan   
1039559                               0     5      nan        nan   
1039560                               0     5      nan        nan   
1039561                               0     5      nan        nan   
1039562                               0     5      nan        nan   
1039563                               0     5      nan        nan   
1039564                               0     5      nan        nan   
1039565                               0     5      nan        nan   
1039566                               0     5      nan        nan   
1039567                               0     5      nan        nan   
1039568                               1     5      nan 

In [13]:
result_features['abs_chng_timestamp_1']

{115: {'lag_timestamp_1': 668090043},
 124: {'lag_timestamp_1': 571323},
 2746: {'lag_timestamp_1': 835457},
 5382: {'lag_timestamp_1': 2101551456},
 8623: {'lag_timestamp_1': 862338736},
 8701: {'lag_timestamp_1': 1571291},
 12741: {'lag_timestamp_1': 4465043002},
 13134: {'lag_timestamp_1': 18122046414},
 24418: {'lag_timestamp_1': 14243735782},
 24600: {'lag_timestamp_1': 1550831},
 32421: {'lag_timestamp_1': 44680547},
 40828: {'lag_timestamp_1': 6376896727},
 44331: {'lag_timestamp_1': 4661118198},
 45001: {'lag_timestamp_1': 195681},
 46886: {'lag_timestamp_1': 22050635752},
 50132: {'lag_timestamp_1': 2308306376},
 51285: {'lag_timestamp_1': 871989},
 53842: {'lag_timestamp_1': 723461},
 81002: {'lag_timestamp_1': 447505},
 81429: {'lag_timestamp_1': 718663},
 91216: {'lag_timestamp_1': 18804556370},
 99521: {'lag_timestamp_1': 941291},
 107002: {'lag_timestamp_1': 2831179364},
 108310: {'lag_timestamp_1': 1280931},
 128919: {'lag_timestamp_1': 646286},
 137455: {'lag_timestamp_

In [14]:
len(result_features['abs_chng_timestamp_1']),\
len(result_features['abs_chng_timestamp_2']),\
len(result_features['abs_chng_timestamp_3']),

(393656, 393656, 393656)

In [15]:
result_fillna['abs_chng_timestamp_1'], \
result_fillna['abs_chng_timestamp_2'], \
result_fillna['abs_chng_timestamp_3']

(34874.0, 92307.0, 155094.0)

# User mean features

 'user_mean_answered_correctly',
 
 'user_sum_answered_correctly'
 
 'user_he_mean_answered_correctly',
 
 'user_he_mean_prior_question_elapsed_time',

In [16]:
user_mean_answered_correctly = aggregate(
    vitrine,
    groupby_list=["user_id"],
    agg_func=["sum", "count"],
    name_list=["user_sum_", "user_count_"],
    target_field="answered_correctly",
)

user_mean_answered_correctly, result_fillna = get_contnt_prod(
    result_fillna, 
    user_mean_answered_correctly, 
    prename = 'user',
    keys = ['user_id']
)

{'content_he_mean_answered_correctly': 0.7221108884435538, 'content_he_sum_answered_correctly': 323.0, 'content_he_count_answered_correctly': 525.0, 'content_mean_answered_correctly': 0.7342154131847726, 'content_sum_answered_correctly': 3236.0, 'content_count_answered_correctly': 4732.0, 'content_he_part_mean_answered_correctly': 0.6309980073399225, 'content_he_part_sum_answered_correctly': 2117852.5, 'content_he_part_count_answered_correctly': 3642251.0, 'prior_question_elapsed_time': 21000.0, 'abs_chng_timestamp_1': 34874.0, 'abs_chng_timestamp_2': 92307.0, 'abs_chng_timestamp_3': 155094.0, 'user_mean_answered_correctly': 0.5714285714285714, 'user_sum_answered_correctly': 23.0, 'user_count_answered_correctly': 40.0}
[(115, {'user_sum_answered_correctly': 32, 'user_count_answered_correctly': 46}), (124, {'user_sum_answered_correctly': 7, 'user_count_answered_correctly': 30}), (2746, {'user_sum_answered_correctly': 11, 'user_count_answered_correctly': 19}), (5382, {'user_sum_answered_

In [17]:
user_he_mean_answered_correctly = aggregate(
    vitrine,
    groupby_list=["user_id", "prior_question_had_explanation"],
    agg_func=["sum", "count"],
    name_list=["user_he_sum_", "user_he_count_"],
    target_field="answered_correctly",
)

user_he_mean_answered_correctly, result_fillna = get_contnt_prod(
    result_fillna, 
    user_he_mean_answered_correctly, 
    prename = 'user_he',
    keys = ["user_id", "prior_question_had_explanation"]
)

{'content_he_mean_answered_correctly': 0.7221108884435538, 'content_he_sum_answered_correctly': 323.0, 'content_he_count_answered_correctly': 525.0, 'content_mean_answered_correctly': 0.7342154131847726, 'content_sum_answered_correctly': 3236.0, 'content_count_answered_correctly': 4732.0, 'content_he_part_mean_answered_correctly': 0.6309980073399225, 'content_he_part_sum_answered_correctly': 2117852.5, 'content_he_part_count_answered_correctly': 3642251.0, 'prior_question_elapsed_time': 21000.0, 'abs_chng_timestamp_1': 34874.0, 'abs_chng_timestamp_2': 92307.0, 'abs_chng_timestamp_3': 155094.0, 'user_mean_answered_correctly': 0.5714285714285714, 'user_sum_answered_correctly': 23.0, 'user_count_answered_correctly': 40.0, 'user_he_mean_answered_correctly': 0.5517241379310345, 'user_he_sum_answered_correctly': 10.0, 'user_he_count_answered_correctly': 23.0}
[((115, 0), {'user_he_sum_answered_correctly': 27, 'user_he_count_answered_correctly': 40}), ((115, 1), {'user_he_sum_answered_correct

In [18]:
user_he_mean_prior_question_elapsed_time = aggregate(
    vitrine,
    groupby_list=["user_id", "prior_question_had_explanation"],
    agg_func=["sum", "count"],
    name_list=["user_he_sum_", "user_he_count_"],
    target_field="prior_question_elapsed_time",
)

user_he_mean_prior_question_elapsed_time, result_fillna = get_contnt_prod(
    result_fillna, 
    user_he_mean_prior_question_elapsed_time, 
    prename = 'user_he',
    keys = ["user_id", "prior_question_had_explanation"],
    target = "prior_question_elapsed_time",
)

{'content_he_mean_answered_correctly': 0.7221108884435538, 'content_he_sum_answered_correctly': 323.0, 'content_he_count_answered_correctly': 525.0, 'content_mean_answered_correctly': 0.7342154131847726, 'content_sum_answered_correctly': 3236.0, 'content_count_answered_correctly': 4732.0, 'content_he_part_mean_answered_correctly': 0.6309980073399225, 'content_he_part_sum_answered_correctly': 2117852.5, 'content_he_part_count_answered_correctly': 3642251.0, 'prior_question_elapsed_time': 21000.0, 'abs_chng_timestamp_1': 34874.0, 'abs_chng_timestamp_2': 92307.0, 'abs_chng_timestamp_3': 155094.0, 'user_mean_answered_correctly': 0.5714285714285714, 'user_sum_answered_correctly': 23.0, 'user_count_answered_correctly': 40.0, 'user_he_mean_answered_correctly': 0.5517241379310345, 'user_he_sum_answered_correctly': 10.0, 'user_he_count_answered_correctly': 23.0, 'user_he_mean_prior_question_elapsed_time': 23000.0, 'user_he_sum_prior_question_elapsed_time': 480663.0, 'user_he_count_prior_questio

In [19]:
result_features['user_mean_answered_correctly'] = user_mean_answered_correctly
result_features['user_he_mean_answered_correctly'] = user_he_mean_answered_correctly
result_features['user_he_mean_prior_question_elapsed_time'] = user_he_mean_prior_question_elapsed_time

user_he_ucount_part = cummulative cumcount_unique

{prename}_ucount_{target} -> prename == user, target == part

количество уникальных part на пару (user_id, prior_question_had_explanation)

In [20]:
user_he_ucount_part = aggregate(
    vitrine,
    groupby_list=["user_id", "prior_question_had_explanation"],
    agg_func=["nunique"],
    name_list=["user_he_ucount_"],
    target_field="part",
)

user_he_ucount_part = df_features_to_dict(
    user_he_ucount_part,
    ["user_id", "prior_question_had_explanation"]
)

In [21]:
user_he_ucount_part

{(115, 0): {'user_he_ucount_part': 5},
 (115, 1): {'user_he_ucount_part': 1},
 (124, 0): {'user_he_ucount_part': 7},
 (2746, 0): {'user_he_ucount_part': 2},
 (2746, 1): {'user_he_ucount_part': 1},
 (5382, 0): {'user_he_ucount_part': 3},
 (5382, 1): {'user_he_ucount_part': 3},
 (8623, 0): {'user_he_ucount_part': 2},
 (8623, 1): {'user_he_ucount_part': 4},
 (8701, 0): {'user_he_ucount_part': 2},
 (8701, 1): {'user_he_ucount_part': 1},
 (12741, 0): {'user_he_ucount_part': 3},
 (12741, 1): {'user_he_ucount_part': 4},
 (13134, 0): {'user_he_ucount_part': 3},
 (13134, 1): {'user_he_ucount_part': 6},
 (24418, 0): {'user_he_ucount_part': 7},
 (24418, 1): {'user_he_ucount_part': 7},
 (24600, 0): {'user_he_ucount_part': 7},
 (24600, 1): {'user_he_ucount_part': 2},
 (32421, 0): {'user_he_ucount_part': 7},
 (40828, 0): {'user_he_ucount_part': 7},
 (40828, 1): {'user_he_ucount_part': 2},
 (44331, 0): {'user_he_ucount_part': 3},
 (44331, 1): {'user_he_ucount_part': 6},
 (45001, 0): {'user_he_ucount_

In [22]:
result_features['user_he_ucount_part'] = user_he_ucount_part

# User EMA features

 'user_ema_answered_correctly',
    
 'user_he_ema_answered_correctly',

In [4]:
from functools import reduce
def ema(series):
    N = len(series)
    alpha = 2 / (N + 1)
    return reduce(lambda sum_, curr: (1 - alpha) * sum_ + alpha * curr, series)



def ema_vect(data):
    N = data.shape[0]

    alpha = 2 /(N + 1.0)
    alpha_rev = 1-alpha
    
    

    scale = 1/alpha_rev
    n = data.shape[0]

    r = np.arange(n)
    scale_arr = scale**r
    offset = data[0]*alpha_rev**(r+1)
    pw0 = alpha*alpha_rev**(n-1)

    mult = data*pw0*scale_arr
    cumsums = mult.cumsum()
    out = offset + cumsums*scale_arr[::-1]
    print(out)
    return out

In [5]:
user_ema_answered_correctly = get_answered_as_arr(
    vitrine,
    agg_fields = ['user_id']
)

user_ema_answered_correctly = df_features_to_dict(
    user_ema_answered_correctly,
    ['user_id']
)

In [6]:
%%time
user_ema = dict()
for user_id in user_ema_answered_correctly:
    user_ema[user_id] = user_ema_answered_correctly[user_id]['answered_correctly']

CPU times: user 120 ms, sys: 6.44 ms, total: 127 ms
Wall time: 125 ms


In [10]:
user_ema

{115: array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0,
        0, 1]),
 124: array([1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0]),
 2746: array([0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1]),
 5382: array([1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0,
        0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1,
        0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
        1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1,
        1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0]),
 8623: array([1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1,
        1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1,
        1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1

In [7]:
user_he_ema_answered_correctly = get_answered_as_arr(
    vitrine,
    agg_fields = ['user_id', 'prior_question_had_explanation']
)

user_he_ema_answered_correctly = df_features_to_dict(
    user_he_ema_answered_correctly,
    ['user_id', 'prior_question_had_explanation']
)

In [8]:
user_he_ema = dict()
for key in user_he_ema_answered_correctly:
    user_id, he = key[0], key[1]
    user_he_ema[(user_id, he)] = user_he_ema_answered_correctly[key]['answered_correctly']

In [9]:
user_he_ema

{(115,
  0): array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1]),
 (115, 1): array([1, 1, 1, 0, 1, 1]),
 (124,
  0): array([1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0]),
 (2746, 0): array([0, 0, 0, 1, 0, 1, 1, 1]),
 (2746, 1): array([1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1]),
 (5382, 0): array([1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1]),
 (5382,
  1): array([0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
        1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
        1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0,
        1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,
        0, 0, 0]),
 (8623, 0): array([1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1]),
 (8623,
  1): array([0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0,

In [16]:
result_features['user_ema_answered_correctly'] = user_ema
result_features['user_he_ema_answered_correctly'] = user_he_ema

In [13]:
result_features = load_obj('result_features_no_att')

In [14]:
result_features.keys()

dict_keys(['content_he_mean_answered_correctly', 'content_mean_answered_correctly', 'content_he_part_count_answered_correctly', 'abs_chng_timestamp_1', 'abs_chng_timestamp_2', 'abs_chng_timestamp_3', 'user_mean_answered_correctly', 'user_he_mean_answered_correctly', 'user_he_mean_prior_question_elapsed_time', 'user_he_ucount_part', 'user_ema_answered_correctly', 'user_he_ema_answered_correctly', 'strike', 'strike_bundle', 'lag_part'])

In [17]:
result_features['user_ema_answered_correctly']

{115: array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0,
        0, 1]),
 124: array([1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0]),
 2746: array([0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1]),
 5382: array([1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0,
        0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1,
        0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
        1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1,
        1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0]),
 8623: array([1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1,
        1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1,
        1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1

In [18]:
save_obj(result_features, 'result_features_no_att_ema')

# User attempt features

'attempt_no_count',

'attempt_no_mean',

In [4]:
attempt_no = aggregate(
    vitrine,
    groupby_list=["user_id", "content_id"],
    agg_func=["sum", "count"],
    name_list=["attempt_no_sum_", "attempt_no_count_"],
    target_field="answered_correctly",
)

'''attempt_no = df_features_to_dict(
    attempt_no,
    ["user_id", "content_id"]
)'''

'attempt_no = df_features_to_dict(\n    attempt_no,\n    ["user_id", "content_id"]\n)'

In [5]:
attempt_no.shape

(86867031, 4)

In [6]:
attempt_no.head()

user_id  content_id  attempt_no_sum_answered_correctly  \
0      115           4                                  0   
1      115           6                                  1   
2      115          25                                  1   
3      115          45                                  0   
4      115          50                                  1   

   attempt_no_count_answered_correctly  
0                                    1  
1                                    1  
2                                    1  
3                                    1  
4                                    1

In [3]:
attempt_no = pd.read_csv('../data/prod/attempt_no.csv')

In [4]:
attempt_no.shape

(86867031, 4)

In [5]:
attempt_no = attempt_no[attempt_no.attempt_no_count_answered_correctly > 1].reset_index(drop=True)

In [6]:
attempt_no.shape

(9467440, 4)

In [7]:
attempt_no['user_id'] = attempt_no['user_id'].astype('int32')
attempt_no['content_id'] = attempt_no['content_id'].astype('int16')

attempt_no['attempt_no_sum_answered_correctly'] = attempt_no['attempt_no_sum_answered_correctly']\
                                                .astype('int8')
attempt_no['attempt_no_count_answered_correctly'] = attempt_no['attempt_no_count_answered_correctly']\
                                                .astype('int8')
attempt_no.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9467440 entries, 0 to 9467439
Data columns (total 4 columns):
 #   Column                               Dtype
---  ------                               -----
 0   user_id                              int32
 1   content_id                           int16
 2   attempt_no_sum_answered_correctly    int8 
 3   attempt_no_count_answered_correctly  int8 
dtypes: int16(1), int32(1), int8(2)
memory usage: 72.2 MB


In [10]:
attempt_no_2 = df_features_to_dict(
    attempt_no,
    ["user_id", "content_id"]
)

In [11]:
attempt_no_2

{(2746, 297): {'attempt_no_sum_answered_correctly': 1,
  'attempt_no_count_answered_correctly': 2},
 (2746, 775): {'attempt_no_sum_answered_correctly': 0,
  'attempt_no_count_answered_correctly': 2},
 (5382, 3859): {'attempt_no_sum_answered_correctly': 1,
  'attempt_no_count_answered_correctly': 2},
 (5382, 4103): {'attempt_no_sum_answered_correctly': 1,
  'attempt_no_count_answered_correctly': 2},
 (5382, 5339): {'attempt_no_sum_answered_correctly': 1,
  'attempt_no_count_answered_correctly': 2},
 (5382, 6106): {'attempt_no_sum_answered_correctly': 1,
  'attempt_no_count_answered_correctly': 2},
 (5382, 7945): {'attempt_no_sum_answered_correctly': 0,
  'attempt_no_count_answered_correctly': 2},
 (8623, 130): {'attempt_no_sum_answered_correctly': 1,
  'attempt_no_count_answered_correctly': 2},
 (8623, 587): {'attempt_no_sum_answered_correctly': 1,
  'attempt_no_count_answered_correctly': 2},
 (8623, 651): {'attempt_no_sum_answered_correctly': 1,
  'attempt_no_count_answered_correctly':

In [12]:
attempt_no_3 = dict()
for key in attempt_no_2:
    value = attempt_no_2[key]
    attempt_no_3[key] = [value['attempt_no_sum_answered_correctly'],
                         value['attempt_no_count_answered_correctly']]

In [13]:
attempt_no_3

{(2746, 297): [1, 2],
 (2746, 775): [0, 2],
 (5382, 3859): [1, 2],
 (5382, 4103): [1, 2],
 (5382, 5339): [1, 2],
 (5382, 6106): [1, 2],
 (5382, 7945): [0, 2],
 (8623, 130): [1, 2],
 (8623, 587): [1, 2],
 (8623, 651): [1, 2],
 (8623, 675): [1, 2],
 (8623, 763): [1, 2],
 (12741, 2907): [2, 2],
 (12741, 2908): [2, 2],
 (12741, 2909): [1, 2],
 (12741, 3117): [0, 2],
 (12741, 3118): [1, 2],
 (12741, 3119): [1, 2],
 (12741, 3270): [0, 2],
 (12741, 3271): [2, 2],
 (12741, 3272): [1, 2],
 (12741, 3483): [2, 2],
 (12741, 3484): [0, 2],
 (12741, 3485): [1, 2],
 (12741, 10716): [2, 2],
 (12741, 10717): [0, 2],
 (12741, 10718): [1, 2],
 (13134, 381): [1, 2],
 (13134, 477): [1, 2],
 (13134, 738): [0, 2],
 (13134, 758): [1, 2],
 (13134, 795): [0, 2],
 (13134, 805): [1, 2],
 (13134, 834): [0, 2],
 (13134, 853): [1, 2],
 (13134, 868): [1, 2],
 (13134, 1047): [0, 2],
 (13134, 1136): [1, 2],
 (13134, 1315): [2, 3],
 (13134, 4109): [1, 2],
 (13134, 4131): [1, 2],
 (13134, 8607): [1, 2],
 (13134, 8740): [

In [14]:
del attempt_no
del attempt_no_2

In [17]:
save_obj(attempt_no_3, 'attempt_no_3')

In [15]:
attempt_no.to_csv('../data/prod/attempt_no.csv', index = False)

In [29]:
attempt_no

{(115, 4): {'attempt_no_sum_answered_correctly': 0,
  'attempt_no_count_answered_correctly': 1},
 (115, 6): {'attempt_no_sum_answered_correctly': 1,
  'attempt_no_count_answered_correctly': 1},
 (115, 25): {'attempt_no_sum_answered_correctly': 1,
  'attempt_no_count_answered_correctly': 1},
 (115, 45): {'attempt_no_sum_answered_correctly': 0,
  'attempt_no_count_answered_correctly': 1},
 (115, 50): {'attempt_no_sum_answered_correctly': 1,
  'attempt_no_count_answered_correctly': 1},
 (115, 51): {'attempt_no_sum_answered_correctly': 1,
  'attempt_no_count_answered_correctly': 1},
 (115, 55): {'attempt_no_sum_answered_correctly': 1,
  'attempt_no_count_answered_correctly': 1},
 (115, 57): {'attempt_no_sum_answered_correctly': 1,
  'attempt_no_count_answered_correctly': 1},
 (115, 100): {'attempt_no_sum_answered_correctly': 1,
  'attempt_no_count_answered_correctly': 1},
 (115, 104): {'attempt_no_sum_answered_correctly': 0,
  'attempt_no_count_answered_correctly': 1},
 (115, 108): {'attem

In [30]:
result_features['attempt_no'] = attempt_no

# User strike features

'strike',

'strike_bundle',

In [31]:
def func_strike(series: pd.Series) -> pd.DataFrame:
    cumsum = 0
    res = []
    dict_ = series.to_dict()
    answ_dict = dict_["answered_correctly"].copy()
    del dict_["answered_correctly"]

    keys = list(dict_.keys())
    try:
        cond_dict = dict_[keys[0]]
    except IndexError:
        cond_dict = None

    for ind, val in answ_dict.items():
        if cond_dict == None:
            if val == 0:
                cumsum = 0
            else:
                cumsum += val
        else:
            try:
                if curr_cond == cond_dict[ind]:
                    if val == 0:
                        cumsum = 0
                    else:
                        cumsum += val
                else:
                    cumsum = 0
                curr_cond = cond_dict[ind]
            except NameError:
                curr_cond = cond_dict[ind]
        res.append(cumsum)

    return pd.DataFrame(data={"col": res})

In [32]:
strike = vitrine.groupby(["user_id"])[["answered_correctly"]].apply(func_strike).reset_index(drop=True)
#del vitrine_copy
strike.columns = ['strike']
strike['user_id'] = vitrine['user_id']
strike['timestamp'] = vitrine['timestamp']

strike = get_user_last_f(
    strike, 
    keys = ['user_id'], 
    nec_field = ['strike']
)

strike = df_features_to_dict(
    strike,
    ["user_id"]
)

In [33]:
strike_bundle = vitrine.groupby(["user_id"])[["answered_correctly", "bundle_id"]].apply(func_strike).reset_index(drop=True)
#del vitrine_copy
strike_bundle.columns = ['strike_bundle']
strike_bundle['user_id'] = vitrine['user_id']
strike_bundle['timestamp'] = vitrine['timestamp']
strike_bundle['bundle_id'] = vitrine['bundle_id']

strike_bundle = get_user_last_f(
    strike_bundle, 
    keys = ['user_id'], 
    nec_field = ['strike_bundle', 'bundle_id']
)

strike_bundle = df_features_to_dict(
    strike_bundle,
    ["user_id"]
)

In [34]:
strike_bundle

{115: {'strike_bundle': 1, 'bundle_id': 3363.0, 'timestamp': 668090043},
 124: {'strike_bundle': 0, 'bundle_id': 7216.0, 'timestamp': 571323},
 2746: {'strike_bundle': 0, 'bundle_id': 484.0, 'timestamp': 835457},
 5382: {'strike_bundle': 0, 'bundle_id': 4070.0, 'timestamp': 2101551456},
 8623: {'strike_bundle': 0, 'bundle_id': 969.0, 'timestamp': 862338736},
 8701: {'strike_bundle': 0, 'bundle_id': 498.0, 'timestamp': 1571291},
 12741: {'strike_bundle': 4, 'bundle_id': 11868.0, 'timestamp': 4465043002},
 13134: {'strike_bundle': 0, 'bundle_id': 11892.0, 'timestamp': 18122046414},
 24418: {'strike_bundle': 0, 'bundle_id': 4163.0, 'timestamp': 14243735782},
 24600: {'strike_bundle': 0, 'bundle_id': 801.0, 'timestamp': 1550831},
 32421: {'strike_bundle': 0, 'bundle_id': 7216.0, 'timestamp': 44680547},
 40828: {'strike_bundle': 0, 'bundle_id': 4782.0, 'timestamp': 6376896727},
 44331: {'strike_bundle': 2, 'bundle_id': 1403.0, 'timestamp': 4661118198},
 45001: {'strike_bundle': 0, 'bundle_i

In [35]:
len(strike_bundle)

393656

In [36]:
len(strike), strike

(393656,
 {115: {'strike': 1, 'timestamp': 668090043},
  124: {'strike': 0, 'timestamp': 571323},
  2746: {'strike': 1, 'timestamp': 835457},
  5382: {'strike': 0, 'timestamp': 2101551456},
  8623: {'strike': 3, 'timestamp': 862338736},
  8701: {'strike': 2, 'timestamp': 1571291},
  12741: {'strike': 4, 'timestamp': 4465043002},
  13134: {'strike': 5, 'timestamp': 18122046414},
  24418: {'strike': 4, 'timestamp': 14243735782},
  24600: {'strike': 1, 'timestamp': 1550831},
  32421: {'strike': 0, 'timestamp': 44680547},
  40828: {'strike': 1, 'timestamp': 6376896727},
  44331: {'strike': 5, 'timestamp': 4661118198},
  45001: {'strike': 0, 'timestamp': 195681},
  46886: {'strike': 0, 'timestamp': 22050635752},
  50132: {'strike': 0, 'timestamp': 2308306376},
  51285: {'strike': 0, 'timestamp': 871989},
  53842: {'strike': 0, 'timestamp': 723461},
  81002: {'strike': 0, 'timestamp': 447505},
  81429: {'strike': 0, 'timestamp': 718663},
  91216: {'strike': 2, 'timestamp': 18804556370},
  99

In [37]:
result_features['strike'] = strike
result_features['strike_bundle'] = strike_bundle

# Generat lag features

lag_expl_and_part_bool

In [38]:
lag_part = get_user_last_f(
    vitrine, 
    keys = ['user_id'], 
    nec_field = ['part']
)
lag_part = df_features_to_dict(
    lag_part,
    ["user_id"]
)

In [39]:
len(lag_part), lag_part

(393656,
 {115: {'part': 4, 'timestamp': 668090043},
  124: {'part': 7, 'timestamp': 571323},
  2746: {'part': 2, 'timestamp': 835457},
  5382: {'part': 5, 'timestamp': 2101551456},
  8623: {'part': 2, 'timestamp': 862338736},
  8701: {'part': 2, 'timestamp': 1571291},
  12741: {'part': 7, 'timestamp': 4465043002},
  13134: {'part': 2, 'timestamp': 18122046414},
  24418: {'part': 5, 'timestamp': 14243735782},
  24600: {'part': 2, 'timestamp': 1550831},
  32421: {'part': 7, 'timestamp': 44680547},
  40828: {'part': 5, 'timestamp': 6376896727},
  44331: {'part': 3, 'timestamp': 4661118198},
  45001: {'part': 7, 'timestamp': 195681},
  46886: {'part': 5, 'timestamp': 22050635752},
  50132: {'part': 5, 'timestamp': 2308306376},
  51285: {'part': 5, 'timestamp': 871989},
  53842: {'part': 7, 'timestamp': 723461},
  81002: {'part': 5, 'timestamp': 447505},
  81429: {'part': 7, 'timestamp': 718663},
  91216: {'part': 5, 'timestamp': 18804556370},
  99521: {'part': 2, 'timestamp': 941291},
  1

In [40]:
result_features['lag_part'] = lag_part

In [41]:
del vitrine

# Fillna

In [20]:
features = pd.read_csv('../data/all_features_ns_full/features.csv')

KeyboardInterrupt: 

In [ ]:
result_fillna = features.median().to_dict()
del features
result_fillna

# Save

In [12]:
import pickle
def load_obj(name):
    with open("../data/prod/" + name + ".pkl", "rb") as f:
        return pickle.load(f)
    
def save_obj(obj, name):
    with open("../data/prod/" + name + ".pkl", "wb") as f:
        pickle.dump(obj, f, protocol = 2)

In [ ]:
#save_obj(result_features, 'result_features')
save_obj(result_fillna, 'result_fillna')

# Save and check

In [3]:
result_features = load_obj('result_features_no_att')

In [4]:
result_features.keys()

dict_keys(['content_he_mean_answered_correctly', 'content_mean_answered_correctly', 'content_he_part_count_answered_correctly', 'abs_chng_timestamp_1', 'abs_chng_timestamp_2', 'abs_chng_timestamp_3', 'user_mean_answered_correctly', 'user_he_mean_answered_correctly', 'user_he_mean_prior_question_elapsed_time', 'user_he_ucount_part', 'user_ema_answered_correctly', 'user_he_ema_answered_correctly', 'strike', 'strike_bundle', 'lag_part'])

In [ ]:
# user
# user_he_mean_answered_correctly
value_dict = F['user_he_mean_answered_correctly'].get(
    (user_id, prior_question_had_explanation))
if value_dict != None:
    user_he_mean_answ[i] = value_dict['user_he_sum_answered_correctly'] / \
                            value_dict['user_he_count_answered_correctly']

# user_mean_answered_correctly
# user_sum_answered_correctly
value_dict = F['user_mean_answered_correctly'].get(user_id)
if value_dict != None:
    user_sum_answ[i] = value_dict['user_sum_answered_correctly']
    user_mean_answ[i] = value_dict['user_sum_answered_correctly'] / \
                        value_dict['user_count_answered_correctly']

# user_he_mean_prior_question_elapsed_time
value_dict = F['user_he_mean_prior_question_elapsed_time'].get(
    (user_id, prior_question_had_explanation))
if value_dict != None:
    user_he_mean_pqet[i] = value_dict['user_he_sum_prior_question_elapsed_time'] / \
                            value_dict['user_he_count_prior_question_elapsed_time']


# user_he_ucount_part
value_dict = F['user_he_ucount_part'].get(
    (user_id, prior_question_had_explanation))
if value_dict != None:
    user_he_ucount_part[i] = value_dict['user_he_ucount_part']

In [10]:
result_features['user_he_mean_prior_question_elapsed_time']

{(115, 0): {'user_he_sum_prior_question_elapsed_time': 781999.0,
  'user_he_count_prior_question_elapsed_time': 39},
 (115, 1): {'user_he_sum_prior_question_elapsed_time': 115000.0,
  'user_he_count_prior_question_elapsed_time': 6},
 (124, 0): {'user_he_sum_prior_question_elapsed_time': 544997.0,
  'user_he_count_prior_question_elapsed_time': 29},
 (2746, 0): {'user_he_sum_prior_question_elapsed_time': 140000.0,
  'user_he_count_prior_question_elapsed_time': 7},
 (2746, 1): {'user_he_sum_prior_question_elapsed_time': 185000.0,
  'user_he_count_prior_question_elapsed_time': 11},
 (5382, 0): {'user_he_sum_prior_question_elapsed_time': 503000.0,
  'user_he_count_prior_question_elapsed_time': 11},
 (5382, 1): {'user_he_sum_prior_question_elapsed_time': 3967000.0,
  'user_he_count_prior_question_elapsed_time': 113},
 (8623, 0): {'user_he_sum_prior_question_elapsed_time': 325000.0,
  'user_he_count_prior_question_elapsed_time': 12},
 (8623, 1): {'user_he_sum_prior_question_elapsed_time': 249

In [11]:
for key, value in result_features['user_he_mean_prior_question_elapsed_time'].items():
    if value['user_he_count_prior_question_elapsed_time'] < 1:
        print(key, value)

(1111114, 0) {'user_he_sum_prior_question_elapsed_time': 0.0, 'user_he_count_prior_question_elapsed_time': 0}
(1880240, 0) {'user_he_sum_prior_question_elapsed_time': 0.0, 'user_he_count_prior_question_elapsed_time': 0}
(2982074, 0) {'user_he_sum_prior_question_elapsed_time': 0.0, 'user_he_count_prior_question_elapsed_time': 0}
(3539494, 0) {'user_he_sum_prior_question_elapsed_time': 0.0, 'user_he_count_prior_question_elapsed_time': 0}
(4068433, 0) {'user_he_sum_prior_question_elapsed_time': 0.0, 'user_he_count_prior_question_elapsed_time': 0}
(4387973, 0) {'user_he_sum_prior_question_elapsed_time': 0.0, 'user_he_count_prior_question_elapsed_time': 0}
(4505650, 0) {'user_he_sum_prior_question_elapsed_time': 0.0, 'user_he_count_prior_question_elapsed_time': 0}
(9302710, 0) {'user_he_sum_prior_question_elapsed_time': 0.0, 'user_he_count_prior_question_elapsed_time': 0}
(10110118, 0) {'user_he_sum_prior_question_elapsed_time': 0.0, 'user_he_count_prior_question_elapsed_time': 0}
(11568330

(754789398, 0) {'user_he_sum_prior_question_elapsed_time': 0.0, 'user_he_count_prior_question_elapsed_time': 0}
(755583688, 0) {'user_he_sum_prior_question_elapsed_time': 0.0, 'user_he_count_prior_question_elapsed_time': 0}
(755664478, 0) {'user_he_sum_prior_question_elapsed_time': 0.0, 'user_he_count_prior_question_elapsed_time': 0}
(755889933, 0) {'user_he_sum_prior_question_elapsed_time': 0.0, 'user_he_count_prior_question_elapsed_time': 0}
(756164268, 0) {'user_he_sum_prior_question_elapsed_time': 0.0, 'user_he_count_prior_question_elapsed_time': 0}
(759272394, 0) {'user_he_sum_prior_question_elapsed_time': 0.0, 'user_he_count_prior_question_elapsed_time': 0}
(759771175, 0) {'user_he_sum_prior_question_elapsed_time': 0.0, 'user_he_count_prior_question_elapsed_time': 0}
(760291064, 0) {'user_he_sum_prior_question_elapsed_time': 0.0, 'user_he_count_prior_question_elapsed_time': 0}
(761168459, 0) {'user_he_sum_prior_question_elapsed_time': 0.0, 'user_he_count_prior_question_elapsed_ti

(1325282739, 0) {'user_he_sum_prior_question_elapsed_time': 0.0, 'user_he_count_prior_question_elapsed_time': 0}
(1325929664, 0) {'user_he_sum_prior_question_elapsed_time': 0.0, 'user_he_count_prior_question_elapsed_time': 0}
(1326324139, 0) {'user_he_sum_prior_question_elapsed_time': 0.0, 'user_he_count_prior_question_elapsed_time': 0}
(1327452964, 0) {'user_he_sum_prior_question_elapsed_time': 0.0, 'user_he_count_prior_question_elapsed_time': 0}
(1327741378, 0) {'user_he_sum_prior_question_elapsed_time': 0.0, 'user_he_count_prior_question_elapsed_time': 0}
(1328523285, 0) {'user_he_sum_prior_question_elapsed_time': 0.0, 'user_he_count_prior_question_elapsed_time': 0}
(1328584764, 0) {'user_he_sum_prior_question_elapsed_time': 0.0, 'user_he_count_prior_question_elapsed_time': 0}
(1328820402, 0) {'user_he_sum_prior_question_elapsed_time': 0.0, 'user_he_count_prior_question_elapsed_time': 0}
(1328913752, 0) {'user_he_sum_prior_question_elapsed_time': 0.0, 'user_he_count_prior_question_e

(1896945153, 0) {'user_he_sum_prior_question_elapsed_time': 0.0, 'user_he_count_prior_question_elapsed_time': 0}
(1898997722, 0) {'user_he_sum_prior_question_elapsed_time': 0.0, 'user_he_count_prior_question_elapsed_time': 0}
(1900102414, 0) {'user_he_sum_prior_question_elapsed_time': 0.0, 'user_he_count_prior_question_elapsed_time': 0}
(1901467154, 0) {'user_he_sum_prior_question_elapsed_time': 0.0, 'user_he_count_prior_question_elapsed_time': 0}
(1901780199, 0) {'user_he_sum_prior_question_elapsed_time': 0.0, 'user_he_count_prior_question_elapsed_time': 0}
(1902780822, 0) {'user_he_sum_prior_question_elapsed_time': 0.0, 'user_he_count_prior_question_elapsed_time': 0}
(1903337834, 0) {'user_he_sum_prior_question_elapsed_time': 0.0, 'user_he_count_prior_question_elapsed_time': 0}
(1904126647, 0) {'user_he_sum_prior_question_elapsed_time': 0.0, 'user_he_count_prior_question_elapsed_time': 0}
(1904221072, 0) {'user_he_sum_prior_question_elapsed_time': 0.0, 'user_he_count_prior_question_e

In [5]:
result_features_no_att = dict()

for key in result_features:
    if key != 'attempt_no':
        result_features_no_att[key] = result_features[key].copy()

In [6]:
result_features_no_att.keys()

dict_keys(['content_he_mean_answered_correctly', 'content_mean_answered_correctly', 'content_he_part_count_answered_correctly', 'abs_chng_timestamp_1', 'abs_chng_timestamp_2', 'abs_chng_timestamp_3', 'user_mean_answered_correctly', 'user_he_mean_answered_correctly', 'user_he_mean_prior_question_elapsed_time', 'user_he_ucount_part', 'user_ema_answered_correctly', 'user_he_ema_answered_correctly', 'strike', 'strike_bundle', 'lag_part'])

In [7]:
save_obj(result_features_no_att, 'result_features_no_att')

In [ ]:
save_obj(result_features[key], key)

In [8]:
del result_features
del result_features_no_att

In [6]:
for key in list(result_features.keys())[-3:]:
    print(key)
    save_obj(result_features[key], key)

strike
strike_bundle
lag_part


In [ ]:
result_features['attempt_no']

In [11]:
result_features['content_he_part_count_answered_correctly']

{(1, 0): {'content_he_part_sum_answered_correctly': 690405,
  'content_he_part_count_answered_correctly': 1108120},
 (1, 1): {'content_he_part_sum_answered_correctly': 4863491,
  'content_he_part_count_answered_correctly': 6346450},
 (2, 0): {'content_he_part_sum_answered_correctly': 530985,
  'content_he_part_count_answered_correctly': 832410},
 (2, 1): {'content_he_part_sum_answered_correctly': 12752354,
  'content_he_part_count_answered_correctly': 17910994},
 (3, 0): {'content_he_part_sum_answered_correctly': 466416,
  'content_he_part_count_answered_correctly': 826731},
 (3, 1): {'content_he_part_sum_answered_correctly': 5594098,
  'content_he_part_count_answered_correctly': 7813176},
 (4, 0): {'content_he_part_sum_answered_correctly': 682846,
  'content_he_part_count_answered_correctly': 1667727},
 (4, 1): {'content_he_part_sum_answered_correctly': 4407838,
  'content_he_part_count_answered_correctly': 6399949},
 (5, 0): {'content_he_part_sum_answered_correctly': 1414331,
  'cont

In [ ]:
df = pd.DataFrame(data = result_features)